# LakeTrace Logger - Safe Usage in Fabric Notebooks

This notebook demonstrates conflict-free import patterns for Microsoft Fabric.

## ✅ Recommended Patterns (No Conflicts)

In [ ]:
# Pattern 1: Safe factory function (RECOMMENDED)
from laketrace import get_logger

log = get_logger("fabric_notebook")
log.info("Notebook started")
log.info("Platform detected", platform="fabric")

In [ ]:
# Pattern 2: Direct class usage
from laketrace import Logger

log = Logger("fabric_notebook")
log.info("Using direct class instantiation")

In [ ]:
# Pattern 3: Import with alias to avoid any conflicts
from laketrace import get_logger as trace_logger

log = trace_logger("fabric_notebook")
log.info("Using aliased import")

In [ ]:
# Pattern 4: Alternative factory function name
from laketrace import create_logger

log = create_logger("fabric_notebook")
log.info("Using create_logger alias")

## ⚠️ Existing Code Compatibility

If your notebook already uses Python's standard logging:

In [ ]:
import logging

# Standard Python logger (existing code)
logger = logging.getLogger(__name__)
logger.info("Standard logging")

# LakeTrace logger (import with different name)
from laketrace import get_logger
trace_log = get_logger("my_job")
trace_log.info("LakeTrace logging")

# Both work independently - no conflicts!

## 📝 Context Binding Example

In [ ]:
from laketrace import get_logger

# Create logger with custom config
log = get_logger(
    "etl_pipeline",
    config={
        "log_dir": "/lakehouse/default/Files/logs",
        "json": True,
        "level": "INFO",
        "rotation_mb": 10,
    }
)

# Bind context for structured logging
stage_log = log.bind(stage="extract", dataset="sales")
stage_log.info("Starting extraction", record_count=10000)

# Nested context
table_log = stage_log.bind(table="fact_sales")
table_log.info("Processing table")

## 🔍 Runtime Detection

In [ ]:
from laketrace import get_logger, detect_runtime

# Detect platform automatically
runtime = detect_runtime()
print(f"Platform: {runtime.platform.value}")
print(f"Runtime Type: {runtime.runtime_type.value}")
print(f"Hostname: {runtime.hostname}")

# Logger includes this context automatically
log = get_logger("my_job")
log.info("Runtime context included in logs")

## 📤 Upload Logs to Lakehouse (End of Notebook)

In [ ]:
from laketrace import get_logger
from datetime import datetime

log = get_logger("my_notebook")

# ... do work ...
log.info("Processing completed")

# At end of notebook, upload logs
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
target_path = f"Files/logs/my_notebook_{timestamp}.log"

success = log.upload_log_to_lakehouse(target_path)
if success:
    print(f"✅ Logs uploaded to: {target_path}")
else:
    print("⚠️ Log upload failed (check local file)")

## 🛡️ Safety Features

### No Conflicts
- `get_logger()` doesn't conflict with `logging.getLogger()`
- Can coexist with standard Python logging
- Won't override existing `logger` variables

### Notebook Re-execution Safe
- Handlers automatically reset on re-init
- No duplicate log entries
- Thread-safe for concurrent execution

### Spark-Safe
- Designed for driver-only logging
- No remote file writes during execution
- Upload only at end-of-run

## 📊 View Logs

In [ ]:
from laketrace import get_logger

log = get_logger("my_job")

# Print last 50 lines of log
log.tail(50)